In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
from statistics import mean 
from scipy.stats import percentileofscore as score
from nsetools import Nse
nse = Nse()

In [2]:
names = [i for i in nse.get_stock_codes()]
final_stock_names = []
try:
    for i in range(1,len(names)):
        if(nse.get_quote(names[i])):
            final_stock_names.append(names[i])
        else:
            continue
except:
    print(True)
    

True


In [3]:
def get_change(first, second):
    diff = second - first
    change = 0
    try:
        if diff > 0:
            change = (diff / first) * 100
        elif diff < 0:
            diff = first - second
            change = -((diff / first) * 100)
    except ZeroDivisionError:
        return float('inf')
    return change

In [4]:
my_columns = ['Name','Price','Previous Price','Previous Return in Percentile','Today Price','Today Return in Percentile','Average Price','Average Return in Percentile','Current Returns','Shares_To_Buy','HQM Score']
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in final_stock_names:
        Data = nse.get_quote(stock)
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            Data['symbol'],
            Data['lastPrice'],
            Data['previousClose'],
            'N/A',
            Data['open'],
            'N/A',
            Data['averagePrice'],
            'N/A',
            float(Data['pChange']),
            'N/A',
            'N/A'

        ],
            index = my_columns
        ),
            ignore_index = True
        )


In [5]:
# final_dataframe.sort_values('Current Returns',ascending = False, inplace = True)
# final_dataframe.reset_index(inplace = True)
final_dataframe

,Name,Price,Previous Price,Previous Return in Percentile,Today Price,Today Return in Percentile,Average Price,Average Return in Percentile,Current Returns,Shares_To_Buy,HQM Score
0,20MICRONS,67.30,68.30,N/A,68.30,N/A,67.98,N/A,-1.46,N/A,N/A
1,21STCENMGM,17.15,17.35,N/A,17.00,N/A,17.25,N/A,-1.15,N/A,N/A
2,360ONE,1777.00,1776.70,N/A,1785.00,N/A,1780.74,N/A,0.02,N/A,N/A
3,3IINFOLTD,32.85,34.20,N/A,34.10,N/A,33.17,N/A,-3.95,N/A,N/A
4,3MINDIA,22130.00,21853.65,N/A,22000.00,N/A,21901.13,N/A,1.26,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
79,ALLSEC,474.00,484.30,N/A,484.10,N/A,475.28,N/A,-2.13,N/A,N/A
80,ALMONDZ,63.50,63.50,N/A,63.95,N/A,64.05,N/A,0.00,N/A,N/A
81,ALOKINDS,12.80,12.20,N/A,12.80,N/A,12.80,N/A,4.92,N/A,N/A
82,ALPA,57.80,59.05,N/A,61.95,N/A,58.54,N/A,-2.12,N/A,N/A


In [6]:
def Calculating():
    global amount
    amount = input("Enter the total amount you want to purchase: ")
    try:
        float(amount)
    except ValueError:
        print("That's not a number!\n Please try again: ")
        amount = input("Enter the amount: ")
        float(amount)
Calculating()
size = float(amount)/len(final_dataframe.index)

In [7]:
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Shares_To_Buy'] = math.floor(size/final_dataframe.loc[i,'Price'])



In [8]:
time_periods = [
    'Previous',
    'Today',
    'Average'
]

for row in final_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price'
        percentile_col = f'{time_period} Return in Percentile'
        final_dataframe.loc[row,percentile_col] = score(final_dataframe[change_col],final_dataframe.loc[row,change_col])

final_dataframe

,Name,Price,Previous Price,Previous Return in Percentile,Today Price,Today Return in Percentile,Average Price,Average Return in Percentile,Current Returns,Shares_To_Buy,HQM Score
0,20MICRONS,67.30,68.30,34.52381,68.30,34.52381,67.98,34.52381,-1.46,176,N/A
1,21STCENMGM,17.15,17.35,11.904762,17.00,13.095238,17.25,13.095238,-1.15,694,N/A
2,360ONE,1777.00,1776.70,90.47619,1785.00,90.47619,1780.74,90.47619,0.02,6,N/A
3,3IINFOLTD,32.85,34.20,21.428571,34.10,22.619048,33.17,22.619048,-3.95,362,N/A
4,3MINDIA,22130.00,21853.65,100.0,22000.00,100.0,21901.13,100.0,1.26,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...
79,ALLSEC,474.00,484.30,69.047619,484.10,70.238095,475.28,70.238095,-2.13,25,N/A
80,ALMONDZ,63.50,63.50,33.333333,63.95,33.333333,64.05,33.333333,0.00,187,N/A
81,ALOKINDS,12.80,12.20,8.333333,12.80,9.52381,12.80,9.52381,4.92,930,N/A
82,ALPA,57.80,59.05,29.761905,61.95,30.952381,58.54,30.952381,-2.12,205,N/A


In [9]:
for row in final_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(final_dataframe.loc[row,f'{time_period} Return in Percentile'])
    final_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)

In [10]:
final_dataframe

,Name,Price,Previous Price,Previous Return in Percentile,Today Price,Today Return in Percentile,Average Price,Average Return in Percentile,Current Returns,Shares_To_Buy,HQM Score
0,20MICRONS,67.30,68.30,34.52381,68.30,34.52381,67.98,34.52381,-1.46,176,34.52381
1,21STCENMGM,17.15,17.35,11.904762,17.00,13.095238,17.25,13.095238,-1.15,694,12.698413
2,360ONE,1777.00,1776.70,90.47619,1785.00,90.47619,1780.74,90.47619,0.02,6,90.47619
3,3IINFOLTD,32.85,34.20,21.428571,34.10,22.619048,33.17,22.619048,-3.95,362,22.222222
4,3MINDIA,22130.00,21853.65,100.0,22000.00,100.0,21901.13,100.0,1.26,0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
79,ALLSEC,474.00,484.30,69.047619,484.10,70.238095,475.28,70.238095,-2.13,25,69.84127
80,ALMONDZ,63.50,63.50,33.333333,63.95,33.333333,64.05,33.333333,0.00,187,33.333333
81,ALOKINDS,12.80,12.20,8.333333,12.80,9.52381,12.80,9.52381,4.92,930,9.126984
82,ALPA,57.80,59.05,29.761905,61.95,30.952381,58.54,30.952381,-2.12,205,30.555556


In [11]:
final_dataframe.sort_values('HQM Score', ascending=False,inplace=True)
final_dataframe

,Name,Price,Previous Price,Previous Return in Percentile,Today Price,Today Return in Percentile,Average Price,Average Return in Percentile,Current Returns,Shares_To_Buy,HQM Score
4,3MINDIA,22130.00,21853.65,100.0,22000.00,100.0,21901.13,100.0,1.26,0,100.0
23,ABBOTINDIA,20501.00,20154.60,98.809524,20232.55,98.809524,20424.40,98.809524,1.72,0,98.809524
76,ALKEM,3270.00,3297.35,97.619048,3297.00,97.619048,3275.12,97.619048,-0.83,3,97.619048
22,ABB,3110.00,3142.00,96.428571,3149.70,96.428571,3105.75,96.428571,-1.02,3,96.428571
58,AIAENG,2671.00,2720.35,95.238095,2700.05,95.238095,2684.65,95.238095,-1.81,4,95.238095
...,...,...,...,...,...,...,...,...,...,...,...
71,ALANKIT,8.60,8.85,5.952381,8.95,7.142857,8.63,7.142857,-2.82,1384,6.746032
9,A2ZINFRA,7.10,7.30,4.761905,7.50,5.952381,6.84,4.761905,-2.74,1676,5.15873
11,AAKASH,7.20,7.05,3.571429,7.20,4.761905,7.20,5.952381,2.13,1653,4.761905
63,AJOONI,4.90,5.00,2.380952,5.00,3.571429,4.96,3.571429,-2.00,2429,3.174603


In [12]:
writer = pd.ExcelWriter('Recommend Stocks based on HQM Score.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommend',index=False)

In [13]:
background_color = '#FFFFFF'
font_color ='#000000'

string_format = writer.book.add_format(
    {
    'font_color':font_color,
    'bg_color':background_color,
    'border': 1
    }
)

Rupee_format = writer.book.add_format(
    {
    'num_format':'INR 00.0',
    'font_color':font_color,
    'bg_color':background_color,
    'border': 1
    }
)
percent_template = writer.book.add_format( {
    'font_color':font_color,
    'bg_color':background_color,
    'border': 1
})


In [14]:
column_formats = {
'A': ['Name', string_format],
'B': ['Price',Rupee_format],
'C': ['Previous Price',Rupee_format],
'D': ['Previous Return in Percentile',percent_template],
'E': ['Today Price',Rupee_format],
'F': ['Today Return in Percentile',percent_template],
'G': ['Average Price',Rupee_format],
'H': ['Average Return in Percentile',percent_template],
'I': ['Current Returns',percent_template],
'J': ['Shares_To_Buy',string_format],
'K': ['HQM Score',string_format]
}

In [15]:
for c in column_formats.keys():
    writer.sheets['Recommend'].set_column(f'{c}:{c}',24, column_formats[c][1])
    writer.sheets['Recommend'].write(f'{c}1', column_formats[c][0],column_formats[c][1])



In [16]:
writer.save()